In [ ]:
import requests
import pandas as pd
import numpy as np

Read the data from API and convert them to json style

In [ ]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()

In [ ]:
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

Covnert the json data to dataFrame to start woking with them.

In [ ]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
event_df = pd.DataFrame(json['events'])
phases_df = pd.DataFrame(json['phases'])
#total_players_df = pd.DataFrame(json['total_players'])
element_stats_df = pd.DataFrame(json['element_stats'])

*   *elements_df* contains the information about players.
*   *teams_df* contains the information about teams.
*   *elements_types_df* contains the information about the positions.

In [ ]:
slim_elements_df = elements_df[['second_name','team','element_type',
                                'selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

An error was occuring so I turned it off here.

In [ ]:
pd.options.mode.chained_assignment = None 

Adding the postion of the players to the main table.

In [ ]:
slim_elements_df['position'] = slim_elements_df['element_type'].map(elements_types_df.set_index('id').singular_name)

In [ ]:
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position
0,Alves Soares,1,2,0.1,42,0,4964,0.0,0,Defender
1,Xhaka,1,3,3.4,51,897,506118,8.8,45,Midfielder
2,Elneny,1,3,0.6,43,90,110131,0.5,2,Midfielder
3,Holding,1,2,0.1,42,11,4663,0.7,3,Defender
4,Partey,1,3,0.5,48,599,53777,5.4,26,Midfielder


Add team names to the main table.

In [ ]:
slim_elements_df['teams'] = slim_elements_df['team'].map(teams_df.set_index('id').name)
slim_elements_df = slim_elements_df.drop('team', axis=1)

In [ ]:
slim_elements_df.head()

,second_name,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,teams
0,Alves Soares,2,0.1,42,0,4964,0.0,0,Defender,Arsenal
1,Xhaka,3,3.4,51,897,506118,8.8,45,Midfielder,Arsenal
2,Elneny,3,0.6,43,90,110131,0.5,2,Midfielder,Arsenal
3,Holding,2,0.1,42,11,4663,0.7,3,Defender,Arsenal
4,Partey,3,0.5,48,599,53777,5.4,26,Midfielder,Arsenal


We would like to sort the values, so we create a table called value and sort the table based on that column.

In [ ]:
slim_elements_df['value'] = slim_elements_df.value_season.astype(float)

In [ ]:
slim_elements_df.sort_values('value',ascending=False).head(15)

,second_name,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,teams,value
616,Malheiro de Sá,1,7.1,51,900,547400,10.8,55,Goalkeeper,Wolves,10.8
23,Saliba,2,29.0,51,900,3219171,10.0,51,Defender,Arsenal,10.0
460,Trippier,2,56.9,56,885,4726802,9.8,55,Defender,Newcastle,9.8
469,Schär,2,10.8,48,802,1339373,9.4,45,Defender,Newcastle,9.4
95,Raya Martin,1,6.9,45,900,545578,9.1,41,Goalkeeper,Brentford,9.1
479,Pope,1,20.9,53,900,2062933,9.1,48,Goalkeeper,Newcastle,9.1
8,White,2,5.9,45,809,402498,9.1,41,Defender,Arsenal,9.1
472,Almirón Rejala,3,7.2,52,838,866160,9.0,47,Midfielder,Newcastle,9.0
94,Toney,4,24.9,73,900,3498732,8.9,65,Forward,Brentford,8.9
1,Xhaka,3,3.4,51,897,506118,8.8,45,Midfielder,Arsenal,8.8


Now we would like to delete all of the players who have zero minutes of playing.

In [ ]:
slim_elements_df = slim_elements_df.loc[slim_elements_df.value > 0]

We look at the average value of each poisiton.

In [ ]:
pivot=slim_elements_df.pivot_table(index='position',values='value',aggfunc=np.mean).reset_index()
pivot.sort_values('value',ascending=False)

,position,value
2,Goalkeeper,5.920000
0,Defender,3.326415
3,Midfielder,3.255838
1,Forward,3.071698


We look at the average value of each team.

In [ ]:
pivot=slim_elements_df.pivot_table(index='teams',values='value',aggfunc=np.mean).reset_index()
pivot.sort_values('value',ascending=False).head(7)

,teams,value
0,Arsenal,4.723810
14,Newcastle,4.454545
12,Man City,4.440000
17,Spurs,4.320000
4,Brighton,4.073684
3,Brentford,3.658333
10,Leicester,3.421053


We can also look at the players in different positions

In [ ]:
fwd_df = slim_elements_df.loc[slim_elements_df.position == 'Forward']
mid_df = slim_elements_df.loc[slim_elements_df.position == 'Midfielder']
def_df = slim_elements_df.loc[slim_elements_df.position == 'Defender']
goal_df = slim_elements_df.loc[slim_elements_df.position == 'Goalkeeper']

In [ ]:
goal_df.sort_values('value',ascending=False).head(10)

,second_name,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,teams,value
616,Malheiro de Sá,1,7.1,51,900,547400,10.8,55,Goalkeeper,Wolves,10.8
479,Pope,1,20.9,53,900,2062933,9.1,48,Goalkeeper,Newcastle,9.1
95,Raya Martin,1,6.9,45,900,545578,9.1,41,Goalkeeper,Brentford,9.1
553,Lloris,1,5.8,55,900,380144,8.4,46,Goalkeeper,Spurs,8.4
505,Henderson,1,12.2,47,900,1072447,8.1,38,Goalkeeper,Nott'm Forest,8.1
13,Ramsdale,1,10.3,49,900,532100,7.8,38,Goalkeeper,Arsenal,7.8
32,Martínez Romero,1,5.6,49,900,335013,7.6,37,Goalkeeper,Aston Villa,7.6
340,Meslier,1,2.7,45,810,176389,7.3,33,Goalkeeper,Leeds,7.3
397,Santana de Moraes,1,14.3,55,900,778092,7.1,39,Goalkeeper,Man City,7.1
363,Ramses Becker,1,13.1,54,810,392413,7.0,38,Goalkeeper,Liverpool,7.0


In [ ]:
mid_df.sort_values('value',ascending=False).head(10)

,second_name,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,teams,value
472,Almirón Rejala,3,7.2,52,838,866160,9.0,47,Midfielder,Newcastle,9.0
1,Xhaka,3,3.4,51,897,506118,8.8,45,Midfielder,Arsenal,8.8
17,Martinelli Silva,3,46.5,67,871,3933175,8.7,58,Midfielder,Arsenal,8.7
561,Højbjerg,3,3.0,55,884,402657,8.5,47,Midfielder,Spurs,8.5
128,Groß,3,19.3,58,752,3339656,8.3,48,Midfielder,Brighton,8.3
76,Billing,3,1.3,53,725,140123,8.1,43,Midfielder,Bournemouth,8.1
133,Trossard,3,18.2,69,738,1960842,8.1,56,Midfielder,Brighton,8.1
11,Saka,3,13.6,79,878,894402,8.0,63,Midfielder,Arsenal,8.0
282,Hoelgebaum Pereira,3,24.7,46,812,1552012,8.0,37,Midfielder,Fulham,8.0
137,Mac Allister,3,6.7,57,810,1056295,7.9,45,Midfielder,Brighton,7.9


In [ ]:
def_df.sort_values('value',ascending=False).head(10)

,second_name,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,teams,value
23,Saliba,2,29.0,51,900,3219171,10.0,51,Defender,Arsenal,10.0
460,Trippier,2,56.9,56,885,4726802,9.8,55,Defender,Newcastle,9.8
469,Schär,2,10.8,48,802,1339373,9.4,45,Defender,Newcastle,9.4
8,White,2,5.9,45,809,402498,9.1,41,Defender,Arsenal,9.1
558,Dier,2,11.1,52,900,1043746,8.7,45,Defender,Spurs,8.7
396,Cancelo,2,53.6,73,826,2469321,8.6,63,Defender,Man City,8.6
14,dos Santos Magalhães,2,10.0,51,900,778967,8.6,44,Defender,Arsenal,8.6
621,Kilman,2,3.3,46,900,332692,7.8,36,Defender,Wolves,7.8
118,Mee,2,3.3,45,848,523961,7.6,34,Defender,Brentford,7.6
304,Castagne,2,2.9,44,741,303457,7.5,33,Defender,Leicester,7.5


In [ ]:
fwd_df.sort_values('value',ascending=False).head(10)

,second_name,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,teams,value
94,Toney,4,24.9,73,900,3498732,8.9,65,Forward,Brentford,8.9
407,Haaland,4,81.6,122,841,4123010,8.5,104,Forward,Man City,8.5
72,Solanke,4,2.8,57,673,283335,7.2,41,Forward,Bournemouth,7.2
270,Mitrović,4,22.6,67,755,3350818,7.2,48,Forward,Fulham,7.2
361,Firmino,4,7.3,80,536,872909,6.9,55,Forward,Liverpool,6.9
25,Fernando de Jesus,4,64.7,80,860,1975669,6.8,54,Forward,Arsenal,6.8
555,Kane,4,22.7,114,886,1712834,6.3,72,Forward,Spurs,6.3
109,Mbeumo,4,1.7,59,753,280792,5.1,30,Forward,Brentford,5.1
317,Daka,4,1.4,57,393,128113,4.9,28,Forward,Leicester,4.9
532,Armstrong,4,1.2,55,624,216747,4.9,27,Forward,Southampton,4.9
